### 輸入所在地獲取現在天氣資訊

In [1]:
def getWeatherData(city, place, current_Time=None):
    import requests
    import json
    import pandas as pd
    from datetime import datetime, timedelta
    import time

    district = {
        '宜蘭縣': '01', '桃園市': '05', '臺北市': '61', '新北市': '69', '臺中市': '73', '臺南市': '77',
        '高雄市': '65', '新竹縣': '09', '苗栗縣': '13', '彰化縣': '17', '南投縣': '21', '雲林縣': '25',
        '嘉義縣': '29', '屏東縣': '33', '花蓮縣': '41', '臺東縣': '37', '澎湖縣': '45', '金門縣': '85',
        '連江縣': '81', '基隆市': '49', '新竹市': '53', '嘉義市': '57'
    }
    url = "https://opendata.cwa.gov.tw/api/v1/rest/datastore/F-D0047-0" + district[city]

    # 資料是每3個小時更新一次，如果沒輸入時間就 default 是現在時間，取離現在時間最近的前一筆資料
    if current_Time is None:
        current_Time = datetime.now()
    current_time_str = current_Time.strftime('%Y-%m-%dT%H:%M:%S')

    params = {
        "Authorization": "CWA-7EBCB64E-EA36-4A13-9D77-82DC3F7E877C",
        "elementName": ['PoP6h', 'WeatherDescription', 'CI', 'T', 'AT'],
        "locationName": [place],
        "timeFrom": current_time_str
    }

    max_retries = 5
    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()  # 若請求失敗會引發 HTTPError
            data = pd.read_json(response.text)
            data = data.loc['locations', 'records'][0]['location']
            info_now = {}
            for element in data[0]['weatherElement']:
                info_now[element['description']] = element['time'][0]['elementValue'][0]['value']
            return info_now
        except requests.exceptions.RequestException as e:
            print(f"網路異常，請耐心等待")
            retries += 1
            time.sleep(2)  # 等待2秒後重試
        except (KeyError, IndexError) as e:
            print(f"Parsing failed: {e}. Retrying ({retries + 1}/{max_retries})...")
            retries += 1
            time.sleep(2)  # 等待2秒後重試

    print("Failed to retrieve data after multiple retries.")
    return None
print(getWeatherData('臺北市', '大安區'))

C:\Users\chen\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


{'體感溫度': '36', '溫度': '31', '舒適度指數': '29', '天氣預報綜合描述': '晴。降雨機率 20%。溫度攝氏31度。悶熱。偏東風 平均風速<= 1級(每秒1公尺)。相對濕度79%。', '6小時降雨機率': '20'}


C:\Users\chen\AppData\Local\Temp\ipykernel_21280\2665517621.py:34: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_json(response.text)


### 天氣 Rule-Base Model

https://health.businessweekly.com.tw/article/ARTL003004286

以體感溫度為判斷標準:
    
* 在極端冷熱的情況下只推出符合該溫度的衣服條件
* 在中間的溫度 (14~23) 考量因人差異的可能性較大，因此會推出該溫度加減附近的溫度衣服條件
* 問題: 冬天衣服的選項偏少

output:
{溫度:[[衣服種類][衣服材質]]}

Ex: {35: [['trousers', 'dress', 'vest_dress', 'shorts', 'skirt'], ['Cotton', 'Linen', 'Linen-blend', 'Satin', 'Denim']]}


In [4]:
#Range
#透過用Rule-Base的方式篩選出符合 attribute 及 categories 標籤的服飾，再進行配對

def weather_rule_Base(input_data):
    
    body_temp = int(input_data['體感溫度'])
    
    candidate_attribute=[]
    candidate_cloth_type=[]
    Final_recommend={}
    Note=[]
    
    breatheble_texture=['Cotton','Linen','Linen-blend','Satin','Denim']
    thermal_texture=['Velvet','Cotton','Woven','Crochet','Faux-fur','Faux-leather']
    Clothe_type={1:['dress','skirt','vest_dress','shorts','trousers'],2:['dress','skirt','vest_dress','shorts','trousers'],3:['long_sleeve_dress','short_sleeve_outwear','trouser','shorts'],4:['long_sleeved_shirt','trouser'],5:['long_sleeved_shirt','trouser'],6:['long_sleeved_shirt','trouser'],7:['long_sleeved_shirt','trouser'],8:['long_sleeved_shirt','trouser']}
    Fabric_type={0:['Cotton','Denim','Linen','Linen-blend'],1:['Cotton','Chiffon','Sateen'],2:['Cotton','Knit','Nylon'],3:['Crochet','Cotton','Woven','Faux-fur'],4:['Cotton','Leather','Neoprene'],5:['Cotton','Nylon','Faux-suede'],6:['Cotton','Woven'],7:['Cotton','Faux-fur']}
    
    #如果是極端熱的情況 -> 材質選:透氣，服裝種類選:1度C
    if (body_temp >= 27):
        candidate_attribute.extend(breatheble_texture)
        candidate_cloth_type += list(set(Clothe_type[1] + Clothe_type[2]))
        Final_recommend[body_temp]=[candidate_cloth_type,candidate_attribute]
    #如果是極端冷的情況 -> 材質選:保暖, 服裝種類選 7~8 度C, 建議洋蔥式穿搭(所以我額外加了發熱衣)                                      
    elif (body_temp <= 4):
        candidate_attribute.extend(thermal_texture)
        candidate_cloth_type+=list(set(Clothe_type[6]+Clothe_type[7]+Clothe_type[8]))
        Final_recommend[body_temp]=[candidate_cloth_type,candidate_attribute]
        Note.extend(['Heat-generating clothing','underwear','down jacket'])
    #正常情況
    else:
        # 比較偏熱的情況下(23~26)，材質的選擇於0-2度C,間度1度C
        if (body_temp >= 23):
            target = 27-body_temp
            candidate_cloth_type.extend(Clothe_type[target])
            for i in range(0,3):
                Final_recommend[body_temp+i]=[candidate_cloth_type,candidate_attribute[i]]
        
        # 比較微涼的情況下(14~22)，材質的選擇於2-7度C,間度2度C
        elif (body_temp >= 14 and body_temp < 23):
            #定義一個正負三度的範圍

            temp_rang = [body_temp - 3, body_temp, body_temp + 3]
            temp_rang = sorted(list(set(value for value in temp_rang if value >= 14)))
            
            three_set_attribute=[]
            for i in range(2,8,2):
                three_set_attribute.append(list(set(Fabric_type[i] + Fabric_type[i + 1])))
            three_set_attribute=three_set_attribute[::-1]
            for i in range(0,len(temp_rang)):
                target=min(8,27-temp_rang[i])
                print(three_set_attribute)
                Final_recommend[temp_rang[i]]=[Clothe_type[target],three_set_attribute[i]]
                        
        #偏冷的情況下，(5-13)
        else:
            candidate_attribute.extend(thermal_texture)
            candidate_cloth_type+=Clothe_type[6]+Clothe_type[7]+Clothe_type[8]
            Final_recommend[body_temp]=[candidate_cloth_type,candidate_attribute]
    return Final_recommend
                                           
                                           
inputdata={'體感溫度': '20', '溫度': '28', '舒適度指數': '27', '天氣預報綜合描述': '晴。降雨機率 0%。溫度攝氏28度。悶熱。西北風 平均風速<= 1級(每秒1公尺)。相對濕度86%。', '6小時降雨機率': '0'}
print('當前適合的衣服種類及材質',weather_rule_Base(getWeatherData('臺北市', '大安區')))
print('若為中間溫度推薦的衣服種類及材質',weather_rule_Base(inputdata))


{35: [['trousers', 'dress', 'vest_dress', 'shorts', 'skirt'], ['Cotton', 'Linen', 'Linen-blend', 'Satin', 'Denim']]}


C:\Users\chen\AppData\Local\Temp\ipykernel_12460\2665517621.py:34: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_json(response.text)
